In [1]:
import os
import sys

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

sys.path.append("source")

In [2]:
import numpy as np
import torch
from aim import Repo
from model import VaDE
from sklearn.metrics import (
    adjusted_rand_score,
    calinski_harabasz_score,
    davies_bouldin_score,
    fowlkes_mallows_score,
    silhouette_score,
    v_measure_score,
)
from tqdm import tqdm
from train import get_dataloader
from utils import calc_accuracy

In [3]:
repo = Repo(".")

torch.manual_seed(123)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [4]:
def get_runs(repo, experiment, dataset):
    return list(
        filter(
            lambda run: run.experiment == experiment
            and run["hparams"]["dataset"] == dataset,
            repo.iter_runs(),
        )
    )


def get_model(run, device):
    model = VaDE(
        run["hparams"]["input_dim"],
        run["hparams"]["latent_dim"],
        run["hparams"]["n_clusters"],
    ).to(device)
    model.load_state_dict(torch.load(f"weights/{run.hash}_200.pth", device))
    return model


def calc_outputs_1(model, dataloader, device):
    model.eval()
    outputs = []
    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device)
            y_hat = model.predict(x)
            y_hat = y_hat.numpy(force=True)
            y = y.numpy(force=True)
            outputs.append((y_hat, y))
    y_hat = np.concatenate([output[0] for output in outputs])
    y = np.concatenate([output[1] for output in outputs])
    return y_hat, y


def calc_outputs_2(model, dataloader, device):
    model.eval()
    outputs = []
    with torch.no_grad():
        for x, _ in dataloader:
            x = x.to(device)
            mu_z, log_sigma2_z = model.encoder(x)
            z = model.sample_z(mu_z, log_sigma2_z)
            y_hat = model.predict(x)
            z = z.numpy(force=True)
            y_hat = y_hat.numpy(force=True)
            outputs.append((z, y_hat))
    z = np.concatenate([output[0] for output in outputs])
    y_hat = np.concatenate([output[1] for output in outputs])
    return z, y_hat


def print_results(results):
    for key, value in results.items():
        print(
            f"{key}:\n\t{np.mean(value):.2f}±{np.std(value):.2f} ({np.min(value):.2f} - {np.max(value):.2f})"
        )

## 1. Supervised metrics

### MNIST

In [5]:
dataloader = get_dataloader("mnist", 1_000, 4, device)

In [6]:
results = {}
for run in tqdm(get_runs(repo, "vade", "mnist")):
    model = get_model(run, device)
    y_hat, y = calc_outputs_1(model, dataloader, device)

    results.setdefault("Clustering accuracy", []).append(calc_accuracy(y_hat, y))

    results.setdefault("Adjusted Rand index", []).append(adjusted_rand_score(y, y_hat))
    results.setdefault("V-measure", []).append(v_measure_score(y, y_hat))
    results.setdefault("Fowlkes-Mallows index", []).append(
        fowlkes_mallows_score(y, y_hat)
    )

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:58<00:00,  5.83s/it]


In [7]:
print_results(results)

Clustering accuracy:
	0.81±0.08 (0.64 - 0.93)
Adjusted Rand index:
	0.74±0.08 (0.57 - 0.87)
V-measure:
	0.82±0.04 (0.76 - 0.87)
Fowlkes-Mallows index:
	0.77±0.07 (0.63 - 0.88)


### Fashion MNIST

In [8]:
dataloader = get_dataloader("fashion-mnist", 1_000, 4, device)

In [9]:
results = {}
for run in tqdm(get_runs(repo, "vade", "fashion-mnist")):
    model = get_model(run, device)
    y_hat, y = calc_outputs_1(model, dataloader, device)

    results.setdefault("Clustering accuracy", []).append(calc_accuracy(y_hat, y))

    results.setdefault("Adjusted Rand index", []).append(adjusted_rand_score(y, y_hat))
    results.setdefault("V-measure", []).append(v_measure_score(y, y_hat))
    results.setdefault("Fowlkes-Mallows index", []).append(
        fowlkes_mallows_score(y, y_hat)
    )

100%|██████████| 10/10 [01:04<00:00,  6.40s/it]


In [10]:
print_results(results)

Clustering accuracy:
	0.62±0.02 (0.59 - 0.64)
Adjusted Rand index:
	0.50±0.02 (0.46 - 0.52)
V-measure:
	0.65±0.02 (0.61 - 0.68)
Fowlkes-Mallows index:
	0.55±0.02 (0.52 - 0.58)


## 2. Unsupervised metrics

### MNIST

In [11]:
dataloader = get_dataloader("mnist", 1_000, 4, device)

In [12]:
results = {}
for run in tqdm(get_runs(repo, "vade", "mnist")):
    model = get_model(run, device)
    z, y_hat = calc_outputs_2(model, dataloader, device)

    results.setdefault("Silhouette coefficient", []).append(
        silhouette_score(z, y_hat, sample_size=7_000, random_state=123)
    )
    results.setdefault("Calinski-Harabasz index", []).append(
        calinski_harabasz_score(z, y_hat)
    )
    results.setdefault("Davies-Bouldin index", []).append(
        davies_bouldin_score(z, y_hat)
    )

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [01:20<00:00,  8.04s/it]


In [13]:
print_results(results)

Silhouette coefficient:
	0.22±0.02 (0.19 - 0.25)
Calinski-Harabasz index:
	11139.18±471.74 (10412.61 - 11874.92)
Davies-Bouldin index:
	1.57±0.11 (1.44 - 1.84)


### Fashion MNIST

In [14]:
dataloader = get_dataloader("fashion-mnist", 1_000, 4, device)

In [15]:
results = {}
for run in tqdm(get_runs(repo, "vade", "fashion-mnist")):
    model = get_model(run, device)
    z, y_hat = calc_outputs_2(model, dataloader, device)

    results.setdefault("Silhouette coefficient", []).append(
        silhouette_score(z, y_hat, sample_size=7_000, random_state=123)
    )
    results.setdefault("Calinski-Harabasz index", []).append(
        calinski_harabasz_score(z, y_hat)
    )
    results.setdefault("Davies-Bouldin index", []).append(
        davies_bouldin_score(z, y_hat)
    )

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [01:25<00:00,  8.54s/it]


In [16]:
print_results(results)

Silhouette coefficient:
	0.23±0.03 (0.19 - 0.27)
Calinski-Harabasz index:
	17571.03±697.77 (16144.76 - 18604.26)
Davies-Bouldin index:
	1.57±0.09 (1.43 - 1.75)
